In [15]:
import pandas as pd
import numpy as np
train_op=pd.read_csv(r"D:\DA_competition\DC\data\operation_train.csv")
train_tag=pd.read_csv(r"D:\DA_competition\DC\data\tag_train.csv")
train_tr=pd.read_csv(r"D:\DA_competition\DC\data\transaction_train.csv")

#表连接
train_op=pd.merge(train_op,train_tag,on="UID",how="left")
train_op=train_op.drop_duplicates()
train_tr=pd.merge(train_tr,train_tag,on="UID",how="left")
train_tr=train_tr.drop_duplicates()

test_op=pd.read_csv(r"D:\DA_competition\DC\data\operation_test.csv")
test_tr=pd.read_csv(r"D:\DA_competition\DC\data\transaction_test.csv")
test_op=test_op.drop_duplicates()
test_tr=test_tr.drop_duplicates()

#处理时间
train_op["hour"]=train_op.time.apply(lambda x:x.split(":")[0])
train_tr["hour"]=train_tr.time.apply(lambda x:x.split(":")[0])
test_op["hour"]=test_op.time.apply(lambda x:x.split(":")[0])
test_tr["hour"]=test_tr.time.apply(lambda x:x.split(":")[0])

D:\developsoft\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
train_op["device_code"]=train_op[["device_code1","device_code2","device_code3"]].fillna(method="bfill",axis=1).iloc[:,0]
train_op["ip"]=train_op[["ip1","ip2"]].fillna(method="bfill",axis=1).iloc[:,0]
train_op["ip_sub"]=train_op[["ip1_sub","ip2_sub"]].fillna(method="bfill",axis=1).iloc[:,0]

test_op["device_code"]=test_op[["device_code1","device_code2","device_code3"]].fillna(method="bfill",axis=1).iloc[:,0]
test_op["ip"]=test_op[["ip1","ip2"]].fillna(method="bfill",axis=1).iloc[:,0]
test_op["ip_sub"]=test_op[["ip1_sub","ip2_sub"]].fillna(method="bfill",axis=1).iloc[:,0]

train_tr["device_code"]=train_tr[["device_code1","device_code2","device_code3"]].fillna(method="bfill",axis=1).iloc[:,0]
test_tr["device_code"]=test_tr[["device_code1","device_code2","device_code3"]].fillna(method="bfill",axis=1).iloc[:,0]

In [17]:
#operation表
#1、UID对应mode,success,os,version:cross_type
#transaction表
#1、UID对应channel、amt_src1、trans_type1、amt_src2、trans_type2、market_type：cross_type
op_fields1=["mode","success","os","version"]
tr_fields1=["channel","amt_src1","trans_type1","amt_src2","trans_type2"]
def get_discrete_fts1(train_data,test_data,fields):
    data_one_hot=pd.get_dummies(pd.concat([train_data[fields].applymap(str),test_data[fields].applymap(str)],axis=0),dummy_na=True)
    train_data_one_hot=data_one_hot.iloc[:len(train_data),:].reset_index(drop=True)
    test_data_one_hot=data_one_hot.iloc[len(train_data):,:].reset_index(drop=True)
    train_data_one_hot["UID"]=train_data["UID"]
    test_data_one_hot["UID"]=test_data["UID"]
    train_fts_cross_type=train_data_one_hot.groupby("UID").sum().reset_index(drop=False)
    test_fts_cross_type=test_data_one_hot.groupby("UID").sum().reset_index(drop=False)
    
    print("=====OK======")
    return train_fts_cross_type,test_fts_cross_type


In [18]:
#operation表
# 2.1、UID对应day、time、mode、success、os、version、device1、device2、geo_code种类数
# 2.2、UID对应device_code1、2、3、mac1、2、wifi、ip1、ip2、ip1_sub、ip2_sub种类数
#transaction表
# 2.1、UID对应channel,day,time,amt_src1,amt_src2,merchant,code1,code2,trans_type1,trans_type2,device1,device2,geo_code,market_code,market_type种类数
# 2.2、UID对应ip1、ip1_sub、acc_id1、acc_id2、acc_id3、device_code1、2、3、mac1数
op_fields2=["day","hour","mode","success","os","version","device1","device2","device_code","ip","ip_sub"]
tr_fields2=["channel","day","hour","amt_src1","amt_src2","merchant","trans_type1","trans_type2","device2","ip1","ip1_sub","acc_id1","mac1","device_code"]
def get_kinds_fts2(data,fields):
    fts=data.groupby("UID")[fields].apply(lambda x:x.nunique()).reset_index(drop=False)
    print("=====OK======")
    return fts

In [19]:
#operation表
# 3.1、UID的device_code1、2、3、mac1、2、wifi、ip1、ip2、ip1_sub、ip2_sub对应的用户数量均值、最大值、最小值（因为一个UID可能对应多个连续字段）
# 3.2、UID的device_code1、2、3、mac1、2、wifi、ip1、ip2、ip1_sub、ip2_sub对应的用户羊毛党概率占比均值、最大值、最小值
#transaction表
# 3.1、UID对应ip1、ip1_sub、acc_id1、acc_id2、acc_id3、device_code1、2、3、mac1对应的用户数量均值、最大值、最小值（因为一个UID可能对应多个连续字段）
# 3.2、UID对应ip1、ip1_sub、acc_id1、acc_id2、acc_id3、device_code1、2、3、mac1对应的用户羊毛党概率占比均值、最大值、最小值
op_fields3=["mac1","mac2","wifi","device_code","ip","ip_sub"]
tr_fields3=["ip1","ip1_sub","acc_id1","acc_id2","acc_id3","mac1","device_code"]
# def get_fields_as_key_fts(data,fields):
#     fts_all=pd.DataFrame(data["UID"].unique())
#     fts_all.columns=["UID"]
#     for field in fields:
        
#         d=data[["UID",field,"Tag"]].drop_duplicates().fillna(0).groupby(field)
#         f_count=d.Tag.count()
#         count_1=d.Tag.apply(lambda x:len(x[x==1]))
        
#         df=pd.merge(pd.DataFrame(f_count),pd.DataFrame(count_1),how="left",left_index=True,right_index=True)
#         df.columns=["user_counts","black_counts"]
#         df=df.reset_index(drop=False)
#         df["black_ratio"]=df.black_counts/df.user_counts
        
#         fts_mid=data[["UID",field]].fillna(0)
#         fts_mid=pd.merge(fts_mid,df,how="left",on=field).drop_duplicates()
        
#         fm_group=fts_mid.groupby("UID")
        
#         fts=pd.DataFrame(fm_group.user_counts.mean())
#         fts.columns=[field+"count_mean"]
#         fts[field+"count_max"]=fm_group.user_counts.max()
#         fts[field+"count_min"]=fm_group.user_counts.min()
#         fts[field+"ratio_mean"]=fm_group.black_counts.sum()/fm_group.user_counts.sum()
#         fts[field+"ratio_max"]=fm_group.black_ratio.max()
#         fts[field+"ratio_min"]=fm_group.black_ratio.min()
#         fts=fts.reset_index(drop=False)
#         print(fts)
#         fts_all=pd.merge(fts_all,fts,how="left",on="UID")
#     return fts_all
def get_fields_as_key_fts3(data,fields):
    fts_all=pd.DataFrame(data["UID"].unique())
    fts_all.columns=["UID"]
    for field in fields:
        
        f_count=data[["UID",field]].drop_duplicates().groupby(field).count()
        df=pd.DataFrame(f_count)
        df.columns=["user_counts"]
        df=df.reset_index(drop=False)
        
        fts_mid=data[["UID",field]].fillna(0)
        fts_mid=pd.merge(fts_mid,df,how="left",on=field).drop_duplicates()
        
        fm_group=fts_mid.groupby("UID")
        
        fts=pd.DataFrame(fm_group.user_counts.mean())
        fts.columns=[field+"count_mean"]
        fts[field+"count_max"]=fm_group.user_counts.max()
        fts[field+"count_min"]=fm_group.user_counts.min()
        fts=fts.reset_index(drop=False)
        fts_all=pd.merge(fts_all,fts,how="left",on="UID")
    print("=====OK======")
    return fts_all
        

In [20]:
# operation表
# transaction表
# 4、UID对应样本数
def get_sample_counts_fts4(data):
    fts=pd.DataFrame(data["UID"].value_counts().sort_index()).reset_index(drop=False)
    fts.columns=["UID","UID_sample_counts"]
    print("=====OK======")
    return fts
    

In [21]:
# transaction表
# UID对应trans_amt、bal的和、平均值、最大值、最小值
tr_fields5=["trans_amt","bal"]
def get_tr_money_fts5(data,fields):
    group=data.groupby("UID")[fields]
    fts1=group.apply(lambda x:x.sum())
    fts1.columns=[f+"_sum" for f in fields]
    fts2=group.apply(lambda x:x.max())
    fts2.columns=[f+"_max" for f in fields]
    fts3=group.apply(lambda x:x.min())
    fts3.columns=[f+"_min" for f in fields]
    fts4=group.apply(lambda x:x.mean())
    fts4.columns=[f+"_mean" for f in fields]
    fts=pd.concat([fts1,fts2,fts3,fts4],axis=1)
    fts["UID"]=fts1.index
    print("=====OK======")
    return fts

In [22]:
#transaction表
# 6、divice_code1、2，device1、mac1、geo_code缺失值，羊毛党明显偏高羊毛党：空值数量或者是否包含空值
# ip1,ip1_sub,device2也有20%偏高
#获得空值率
tr_fields6=["device_code1","device_code2","device1","mac1","geo_code","ip1","ip1_sub","device2"]
def get_tr_null_fts6(data,fields):
    group=data.groupby("UID")
    fts=pd.DataFrame(group.apply(lambda x:len(x)))
    fts.columns=["count"]
    for field in fields:
        fts=pd.concat([fts,group[field].apply(lambda x:len(x[x.isnull()]))],axis=1)
    fts=fts.div(fts["count"],axis=0)
    fts=fts.reset_index(drop=False)
    fts=fts.drop(["count"],axis=1)
    print("=====OK======")
    return fts

In [23]:
train_op_fts1,test_op_fts1=get_discrete_fts1(train_op,test_op,op_fields1)
train_tr_fts1,test_tr_fts1=get_discrete_fts1(train_tr,test_tr,tr_fields1)

train_op_fts2=get_kinds_fts2(train_op,op_fields2)
train_tr_fts2=get_kinds_fts2(train_tr,tr_fields2)
train_op_fts3=get_fields_as_key_fts3(train_op,op_fields3)
train_tr_fts3=get_fields_as_key_fts3(train_tr,tr_fields3)
train_op_fts4=get_sample_counts_fts4(train_op)
train_tr_fts4=get_sample_counts_fts4(train_tr)
train_tr_fts5=get_tr_money_fts5(train_tr,tr_fields5)
train_tr_fts6=get_tr_null_fts6(train_tr,tr_fields6)

test_op_fts2=get_kinds_fts2(test_op,op_fields2)
test_tr_fts2=get_kinds_fts2(test_tr,tr_fields2)
test_op_fts3=get_fields_as_key_fts3(test_op,op_fields3)
test_tr_fts3=get_fields_as_key_fts3(test_tr,tr_fields3)
test_op_fts4=get_sample_counts_fts4(test_op)
test_tr_fts4=get_sample_counts_fts4(test_tr)
test_tr_fts5=get_tr_money_fts5(test_tr,tr_fields5)
test_tr_fts6=get_tr_null_fts6(test_tr,tr_fields6)

=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======
=====OK======


In [45]:
train_fts=pd.DataFrame(pd.concat([train_op.UID,train_tr.UID],axis=0).unique())
train_fts.columns=["UID"]
train_fts.sort_values("UID",inplace=True)
train_fts=train_fts.merge(train_op_fts1,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts1,how="left",on="UID")
train_fts=train_fts.merge(train_op_fts2,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts2,how="left",on="UID")
train_fts=train_fts.merge(train_op_fts3,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts3,how="left",on="UID")
train_fts=train_fts.merge(train_op_fts4,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts4,how="left",on="UID")
train_fts=train_fts.merge(train_tr_fts5,how="left",on="UID")
#train_fts=train_fts.merge(train_tr_fts6,how="left",on="UID")

In [46]:
test_fts=pd.read_csv(r"D:\DA_competition\DC\data\sub_sample.csv")
test_fts=test_fts.drop("Tag",axis=1)
test_fts=test_fts.merge(test_op_fts1,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts1,how="left",on="UID")
test_fts=test_fts.merge(test_op_fts2,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts2,how="left",on="UID")
test_fts=test_fts.merge(test_op_fts3,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts3,how="left",on="UID")
test_fts=test_fts.merge(test_op_fts4,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts4,how="left",on="UID")
test_fts=test_fts.merge(test_tr_fts5,how="left",on="UID")
#test_fts=test_fts.merge(test_tr_fts6,how="left",on="UID")

In [47]:
train_fts.fillna(0,inplace=True)
test_fts.fillna(0,inplace=True)
train_fts=pd.merge(train_fts,train_tag,how="left",on="UID")
train_fts.columns

Index(['UID', 'mode_00094ae2a1d62504', 'mode_072eee5c88d380df',
       'mode_08017d2cb28c2348', 'mode_09080b31b40d57e8',
       'mode_0e72a7851fadf00b', 'mode_12845a3fe90eb1de',
       'mode_148dc2618a3a92a4', 'mode_17a6ee1a7b6a02b7',
       'mode_1c341176507fbd9b',
       ...
       'UID_sample_counts_y', 'trans_amt_sum', 'bal_sum', 'trans_amt_max',
       'bal_max', 'trans_amt_min', 'bal_min', 'trans_amt_mean', 'bal_mean',
       'Tag'],
      dtype='object', length=442)

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
X=np.array(train_fts.iloc[:,1:-1])
y=np.array(train_fts.iloc[:,-1])
X_train,X_verify,y_train,y_verify=train_test_split(X,y,test_size=0.2,random_state=0)
X_test=np.array(test_fts)

In [50]:
import lightgbm as lgb
train_set=lgb.Dataset(X_train,y_train)
params={'boosting':'gbdt',
        'objective': 'binary',
        'learning_rate': 0.1,
        'max_depth': 6,
        'num_leaves': 31,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'min_child_weight': 25,
        }
n_rounds=1000
clf=lgb.train(params,train_set,n_rounds)

In [51]:
def tpr_weight_funtion(y_true,y_predict):
    d = pd.DataFrame()
    d['prob'] = list(y_predict)
    d['y'] = list(y_true)
    d = d.sort_values(['prob'], ascending=[0])
    y = d.y
    PosAll = pd.Series(y).value_counts()[1]
    NegAll = pd.Series(y).value_counts()[0]
    pCumsum = d['y'].cumsum()
    nCumsum = np.arange(len(y)) - pCumsum + 1
    pCumsumPer = pCumsum / PosAll
    nCumsumPer = nCumsum / NegAll
    TR1 = pCumsumPer[abs(nCumsumPer-0.001).idxmin()]
    TR2 = pCumsumPer[abs(nCumsumPer-0.005).idxmin()]
    TR3 = pCumsumPer[abs(nCumsumPer-0.01).idxmin()]
    return 0.4 * TR1 + 0.3 * TR2 + 0.3 * TR3


In [52]:
#验证集
y_verify_pre=clf.predict(X_verify)
tpr_weight_funtion(y_verify,y_verify_pre)

0.70233644859813082

In [53]:
#测试集
train_set=lgb.Dataset(X,y)
clf=lgb.train(params,train_set,n_rounds)


In [54]:
y_submit=clf.predict(X_test)

In [55]:
result=pd.concat([test_fts["UID"],pd.Series(y_submit)],axis=1)

In [56]:
result.columns=["UID","Tag"]
result.to_csv(r"D:\DA_competition\DC\data\result_without_6.csv",index=False)

In [57]:
# train_fts.to_csv(r"D:\DA_competition\DC\data\train_fts_without_6.csv",index=False)
# test_fts.to_csv(r"D:\DA_competition\DC\data\test_fts_without_6.csv",index=False)


In [58]:
#减少1：线下0.71507009345794392 线上：0.06081
#减少2：线下：0.74836448598130845 线上：0.00***9
#减少1、6：线下：0.70992990654205612 线上：0.069519
#减少1、6、4：线上：0.04
#减少1、3：线下：0.66635514018691588 线上：0.01
#减少1、4：线下：0.70712616822429908 线上：0.00***6
#减少1、5：线下：0.70864485981308412 线上：0.070254

In [59]:
#one-hot的时候特征的顺序全都不对了，就很烦，训练集测试集应该并在一起进行one-hot cross-type

In [60]:
#修改了代码bug cross-type:线下：0.71343457943925226 线上：0.05

#code1,code2
#删除了空值很多的字段，合并了可合并项，取消了fts3填充0值
#线下：0.71285046728971957，线上：